In [6]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
#sns.set_style("darkgrid")
#sns.set_context("poster")
import sys
import os
from scipy.optimize import newton, minimize, fsolve
from scipy.interpolate import InterpolatedUnivariateSpline, krogh_interpolate, PchipInterpolator, interp1d
import numpy as np
import copy
import re
from pathlib import Path

import neutcurve
from neutcurve.colorschemes import CBMARKERS, CBPALETTE

import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
palette = list(mcolors.TABLEAU_COLORS.keys())
palette.extend(['salmon', 'teal','yellowgreen'])
%matplotlib inline

import random
from pyswarm import pso
import isocor

In [7]:
element_natural_prob = {
    'C': 0.0107,
    'H': 0.000115,
    'N': 0.00364
}

In [8]:
### Adjust for specific label:
def obj_iso(v1, element_natural_prob, element_list, print_sim=False):
    # Adjust:
    element_prob = [element_natural_prob[element_list[0]], element_natural_prob[element_list[1]], v1, v1]

    N_trials = 10000000
    ra = np.random.binomial(1, element_prob, (N_trials, len(element_prob)))
    sim_data = np.bincount(ra.sum(1)) / len(ra) * 100
    # Adjust:
    loss = abs(obs_data[0] - sim_data[0]) + abs(obs_data[1] - sim_data[1]) + abs(obs_data[2] - sim_data[2]) + abs(obs_data[3] - sim_data[3])
    print(v1, loss)
    if print_sim:
        return(sim_data)
    else:
        return(loss)

In [9]:
# Arg 15N2. m+0,1,2,3,4 abundance data:
metabolite = 'Arg'
ion_formula = 'C6H15N4O2'
tracer_element = '15N'
charge = 1
obs_data = {
    0: 0.4235551,
    1: 2.3266937,
    2: 96.6141527,
    3: 0.6350048,
    4: 0.0005937
}
element_list = ['N']*4

In [10]:
# Correct the effect of natural abundance of other elements than carbon.
# No natural abundance correction for the tracer and assume 100% tracer purity
# to avoid multiplication with the purity matrix.
corrector_HR = isocor.mscorrectors.MetaboliteCorrectorFactory(formula=ion_formula, label=metabolite, tracer=tracer_element, tracer_purity=[0, 1], resolution=240000, mz_of_resolution=200, charge=charge, correct_NA_tracer=False, resolution_formula_code='orbitrap')

In [11]:
# Calculate the corrected isotope fraction:
corrected_area, iso_fraction, res, m_enr = corrector_HR.correct([obs_data[i] for i in range(len(obs_data))])

In [14]:
# Update the data:
for i in range(len(obs_data)):
    obs_data[i] = iso_fraction[i] * 100

In [15]:
# Use particle swarm to find best probability to fit the data:
def iso_fun(v1): return(obj_iso(v1, element_natural_prob, element_list))
xopt, fopt = pso(iso_fun, (0.95,), (0.9999999,), swarmsize=20, maxiter=20, minstep=1e-7, minfunc=1e-7)

[0.9756825] 4.900519300000007
[0.95702227] 11.737989300000008
[0.95327397] 13.084479300000009
[0.9674078] 7.979879300000011
[0.96101371] 10.296139299999998
[0.958515] 11.2394593
[0.96196127] 9.939999300000006
[0.99944111] 5.275811300000001
[0.99051635] 1.7510013000000033
[0.96634898] 8.364979300000003
[0.99295446] 2.7091312999999926
[0.97690297] 4.406669300000007
[0.98874246] 1.0380112999999973
[0.95509751] 12.436259300000007
[0.9972157] 4.397361299999984
[0.95254738] 13.334689300000004
[0.98053677] 3.046779299999995
[0.97662145] 4.533899299999996
[0.95798953] 11.401659300000002
[0.97703692] 4.36185930000001
[0.97099581] 6.651839300000004
[0.95] 14.24956930000001
[0.97697547] 4.3858692999999995
[0.99142301] 2.114211299999986
[0.98596419] 0.9750293000000078
[0.95] 14.20965930000001
[0.98375193] 1.8319093000000106
[0.9999999] 5.499771299999988
[0.9999999] 5.499821299999999
[0.95] 14.246879300000003
[0.9999999] 5.499581299999994
[0.9706208] 6.774989300000009
[0.98971098] 1.420241299999991

[0.98610648] 0.937759300000003
[0.98666231] 0.8113738999999893
[0.98625886] 0.875899300000007
[0.9862457] 0.8684693000000113
[0.98633922] 0.8488193000000102
[0.98738872] 0.8160538999999917
[0.98675098] 0.8124138999999863
[0.98672871] 0.811863899999995
[0.98654243] 0.8106238999999908
[0.98650068] 0.8108538999999896
[0.98620932] 0.8921692999999964
[0.98654366] 0.8108538999999874
[0.98646267] 0.8109038999999881
[0.98619629] 0.9064093000000119
[0.98430521] 1.6406293000000032
[0.98680164] 0.8123838999999845
[0.98629868] 0.8524493000000056
[0.98667751] 0.8115839000000004
[0.98654985] 0.8085238999999974
[0.9868738] 0.8110538999999979
[0.98617704] 0.9115692999999999
[0.98681183] 0.8118438999999968
[0.98624697] 0.8846392999999991
[0.98614755] 0.8931393000000072
[0.9862707] 0.8611993000000042
[0.98740716] 0.8151938999999844
[0.98658912] 0.81120389999999
[0.98677192] 0.8105938999999978
[0.9865249] 0.8107338999999935
[0.98664426] 0.8112039000000031
[0.98636934] 0.8176793000000032
[0.98659251] 0.81

In [16]:
# Print probability:
xopt[0], fopt

(0.9865498455996502, 0.8085238999999974)

In [17]:
# Get predictions:
obj_iso(xopt[0], element_natural_prob, element_list, print_sim=True)

0.9865498455996502 0.8100438999999959


array([1.823000e-02, 2.630480e+00, 9.664175e+01, 7.083400e-01,
       1.200000e-03])